# Basic Prompt Engineering (Korean Language)

## Step 1. Prepare Large Language Model (LLM) and Embedding Model 
---

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../utils')
sys.path.append('../templates') 

In [2]:
import time
import sagemaker, boto3, json
import glob
import os
import pandas as pd
import requests
import json
from sagemaker.session import Session
from sagemaker.model import Model
from sagemaker import image_uris, model_uris, script_uris, hyperparameters
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from typing import Any, Dict, List, Optional
from ssm import parameter_store
from termcolor import colored
from common import get_apigateway_url

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name

RESTAPI_ID, URL = get_apigateway_url()
print("RESTAPI_ID = ", RESTAPI_ID)
print("API GATEWAY URL = ", URL)

RESTAPI_ID =  a6uibyuabj
API GATEWAY URL =  https://a6uibyuabj.execute-api.us-west-2.amazonaws.com/api/


In [3]:
MODEL_NAME = "KULLM-12-8B"

LLM_INFO = {
    "LLAMA2-7B": f"{URL}llm/llama2_7b",
    "FALCON-40B": f"{URL}llm/falcon_40b",    
    "KULLM-12-8B": f"{URL}llm/kkulm_12_8b",
}

LLM_URL = LLM_INFO[MODEL_NAME]
EMB_URL = f"{URL}/emb/kosimcse"

HEADERS = {    
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

if 'falcon_40b' in LLM_URL:
    LLM_RESPONSE_KEY = "generated_text"
else:
    LLM_RESPONSE_KEY = "generation"
    
print (f'MODEL_NAME: {MODEL_NAME}\nLLM_URL: {LLM_URL}')    

MODEL_NAME: KULLM-12-8B
LLM_URL: https://a6uibyuabj.execute-api.us-west-2.amazonaws.com/api/llm/kkulm_12_8b


#### Generative Configuration
<img src="https://huggingface.co/blog/assets/02_how-to-generate/sampling_search_with_temp.png"/>

* **max_new_tokens**: The maximum number of tokens to generate. Default value is 20, max value is 512.
* repetition_penalty: Controls the likelihood of repetition, defaults to null.
* seed: The seed to use for random generation, default is null.
* stop: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* **do_sample**: Whether or not to use sampling ; use greedy decoding otherwise. Default value is false.
* **top_k**: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is null, which disables top-k-filtering.
* **top_p**: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to null
* **temperature**: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* best_of: Generate best_of sequences and return the one if the highest token logprobs, default to null.
* details: Whether or not to return details about the generation. Default value is false.
* return_full_text: Whether or not to return the full text or only the generated part. Default value is false.
* truncate: Whether or not to truncate the input to the maximum length of the model. Default value is true.
* typical_p: The typical probability of a token. Default value is null.
* watermark: The watermark to use for the generation. Default value is false.

In [4]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": True,
        "max_new_tokens": 256,
        "temperature": 0.2,
        "top_p": 0.9,
        "return_full_text": False,
        "repetition_penalty": 1.2,
        "presence_penalty": None,
        "eos_token_id": 2,
    }    
}

<br>

## Step 2. Ask a question to LLM without RAG
---
<img src="../images/RAG-Page-1.png"/>

### Simple prompt engineering

In [6]:
from lib_ko import Prompter, get_payload
from langchain.llms import AmazonAPIGateway

llm = AmazonAPIGateway(api_url=LLM_URL, headers=HEADERS)
params = PARAMS[MODEL_NAME]
llm.model_kwargs = params

In [7]:
prompter  = Prompter("kullm")

In [8]:
%%time

payload = {
    "inputs": "Generative AI는",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

 데이터를 기반으로 새로운 정보나 지식을 생성하거나, 기존의 정보 및 지식에 대한 이해와 추론 능력이 있습니다. GPT-3 모델은 텍스트 입력 시 사람과 유사한 수준인 100% 정확도(정확도)로 문장을 생성합니다. 또한 GPT-4 모델은 자연어 처리 기능을 활용하여 사용자가 질문하면 답변할 수 있으며, 이는 인간 언어를 이해하고 응답한다는 것입니다. 이러한 기술들은 현재 많은 기업에서 사용되고 있지만, 아직까지는 제한된 작업만 수행 가능하지만 향후 더 발전될 여지가 많이 남아있다고 합니다.​ ​
CPU times: user 18.7 ms, sys: 700 µs, total: 19.4 ms
Wall time: 10.9 s


In [9]:
%%time

payload = {
    "inputs": """Amazon SageMaker의 주요 기능을 간략하게 설명하는 이메일 메시지

여러분 안녕하세요.

다음 사항을 발표합니다.""",
    "parameters": params
}
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])


저희가 제공한 AWS SageMaker에 대한 자세한 내용은 다음과 같습니다:
SageMaker를 사용하여 생성된 새 S3 버킷, SQL 쿼리 및 기타 데이터 개체와 함께 S3 버킷에서 데이터를 검색할 수 있도록 지원되었으며, 이로 인해 고객들에게 더 많은 유연성과 확장성을 제공했습니다.
또한 SageMaker를 통해 생성된 데이터 개체를 S3 버킷으로 전송하고 S3 버킷에서 데이터를 다시 가져올 수도 있어 고객이 S3 버킷에서 데이터를 저장하고 관리할 때 발생하는 문제를 해결해 주었습니다.
이것은 고객들이 SageMaker를 활용함으로써 얻을 수 있던 이점 중 일부입니다.
고객님께 감사드립니다!
앞서 말씀드린 것처럼 저희는 고객 여러분께서 SageMaker를 활용하면 어떤 이점이 있는지 보여드리기 위해 몇 가지 데모 영상도 준비했으니 참고 바랍니다:
1) SageMaker를 사용하여 생성된 새 S3 버킷, SQL 쿼리 및 기타 데이터 개체와 함께 S3 버
CPU times: user 18.7 ms, sys: 159 µs, total: 18.9 ms
Wall time: 13.2 s


### More complex prompts: Play the role of AWS SA


In [10]:
%%time
instruction = """
당신은 AWS에 대한 경험이 풍부한 솔루션 아키텍트입니다. 고객 요구 사항을 분석하여 잘 설계된 솔루션 아키텍처를 만들어 고객에게 제시합니다. 
아키텍처는 상세하고 친절해야 합니다. 다음과 같은 컨텍스트가 주어집니다.
"""

architect_prompt_template = """
Context:
#요구 사항:
{requirements}
#스케일:
{scale}
#특징:
{features}

AWS의 아키텍처를 기술적으로 자세히 설명하세요.
"""
context = architect_prompt_template.format(
    requirements="컴퓨터 광고 웹사이트", 
    scale="최대 초당 10,000건의 요청을 처리해야 합니다. 전 세계에서 사용할 수 있어야 합니다. 응답 속도가 빨라야 합니다.",
    features="제품을 설명하는 랜딩 페이지. 회사를 설명하는 소개 페이지."
)

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

잘 설계되고 효율적인 Amazon Web Services(AWS) 기반 솔루션 아키텍처로 이 모든 것을 충족시킬 수 있습니다. 먼저, 최대 초당 100만 건의 요청을 처리할 수 있도록 하기 위해 서버리스 컴퓨팅 플랫폼을 도입했습니다. 이를 통해 인프라 비용을 절감하고 트래픽 증가에 따라 확장 및 축소할 수 있으므로 매우 유연해졌습니다. 또한 AWS는 전 세계 사용자들이 액세스할 수 있다는 점도 고려했으며, 여러 지역으로 서비스 범위를 넓히면서 글로벌 배포 기능을 구현했습니다. 마지막으로, 빠른 응답 시간을 보장하려면 AWS의 실시간 인메모리 데이터베이스(ALMDB)를 활용해 대량의 데이터를 빠르게 저장하고 검색할 수 있었습니다.
CPU times: user 17.7 ms, sys: 1.05 ms, total: 18.7 ms
Wall time: 8.63 s


In [11]:
%%time

instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""

payload = get_payload(instruction, context, params)
response = requests.post(url=LLM_URL, headers=HEADERS, json=payload)
print(response.json()[0]["generated_text"])

요약하면, 대규모 언어 모델(LLM) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수의 증가로 인해 대규모 언어 모델(LLM)이 더욱 중요해지고 있으며, 이를 파운데이션 모델 또는 기초 모델이라 부릅니다. 파운데이션 모델은 프롬프트 엔지니어링 기술 덕분에 잘 작동하지만, 작업별로 매우 특화되었거나 다양해서 모델을 추가로 커스터마이즈해야 할 때도 있습니다. 이러한 문제를 해결하기 위해 제안된 방법 중 하나인 파인 튜닝은 모델 가중치 및 기타 매개변수를 수정하여 정확도를 높이지만 비용이 많이 듭니다.
CPU times: user 18.5 ms, sys: 102 µs, total: 18.6 ms
Wall time: 7.24 s


### Applying LangChain
<img src="../images/RAG-Page-2.png"/>

In [12]:
llm.model_kwargs = params
print(llm(payload['inputs']))

대규모 언어 모델(LLM) 분야의 발전과 함께, LLM이 가치 있는 인사이트를 제공하는 데 사용될 수 있는 문제 세트도 점점 늘어나고 있으며, 이를 위해 사전 학습보다 작은 작업별로 추가 학습을 수행하면 정확도를 높일 수 있다고 제안했습니다. 그러나 파인 튜닝 방법은 컴퓨팅 리소스 및 메모리 요구 사항이 높아질 수 있으므로 주의가 필요합니다.


In [13]:
from langchain.prompts import PromptTemplate

instruction = """
당신은 AWS에 대한 경험이 풍부한 솔루션 아키텍트입니다. 고객 요구 사항을 분석하여 잘 설계된 솔루션 아키텍처를 만들어 고객에게 제시합니다. 
아키텍처는 상세하고 친절해야 합니다. 다음과 같은 컨텍스트가 주어집니다.
"""

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt_template,
)

context = prompt.format(
    requirements="자바스크립트로 작성된 외부 웹 애플리케이션, 글로벌 배포",
    scale="분당 평균 500건의 요청, 초당 최대 3000건의 요청까지 이벤트 확장",
    features="모바일 웹사이트, 데스크톱 버전, 자바스크립트"
)

In [14]:
%%time
print(colored(instruction, 'green'))
print(colored(context, 'green'))
payload = get_payload(instruction, context, params)
print(llm(payload['inputs']))


당신은 AWS에 대한 경험이 풍부한 솔루션 아키텍트입니다. 고객 요구 사항을 분석하여 잘 설계된 솔루션 아키텍처를 만들어 고객에게 제시합니다. 
아키텍처는 상세하고 친절해야 합니다. 다음과 같은 컨텍스트가 주어집니다.


Context:
#요구 사항:
자바스크립트로 작성된 외부 웹 애플리케이션, 글로벌 배포
#스케일:
분당 평균 500건의 요청, 초당 최대 3000건의 요청까지 이벤트 확장
#특징:
모바일 웹사이트, 데스크톱 버전, 자바스크립트

AWS의 아키텍처를 기술적으로 자세히 설명하세요.

물론 제가 도와드릴 수 있습니다! 먼저, 이 특정 시나리오에서 어떤 기능 및 서비스가 필요한지 이해해 보겠습니다.

1단계: 필요한 리소스 파악하기
- 모바일 웹 사이트: 현재 사용 중인 모바일 기기(예: 스마트폰)에서 원활하게 액세스할 수 없다면 사용자 경험이 크게 저하될 것입니다. 따라서 모바일 앱 개발이 중요합니다.
- 데스크탑 웹사이트: 데스크톱 웹사이트도 마찬가지로 사용자 친화적이지 않으면 사용자 참여도가 떨어질 가능성이 높으므로 데스크톱용 웹사이트도 개발되어야 합니다.
- JavaScript 기반 웹 애플리케이션: 이 경우 초당 최대 3000개 이상의 요청으로 빠르고 안정적으로 처리할 수 있도록 서버리스 아키텍처가 필요합니다. 또한 글로벌 배포를 지원하려면 여러 지역에 걸쳐 데이터를 분산시켜야 하므로 Amazon Elasticsearch 또는 Apache Kafka와 같이 분산 컴퓨팅 환경을 활용하면 도움이 될 수 있습니다.

2단계: 인프라 요구사항 정의하기
필요한 리소스를 확인했으면 이제 이를 통해 구현할 인프라 요구사항을 정의할 차례입니다. 여기서 고려해야 할 몇
CPU times: user 18 ms, sys: 1.66 ms, total: 19.7 ms
Wall time: 13.5 s


In [15]:
PARAMS = {
    "KULLM-12-8B": {
        "do_sample": True,
        "max_new_tokens": 200,
        "temperature": 0.5,
        "top_p": 0.9,
        "return_full_text": False,
        "repetition_penalty": 1.1,
        "presence_penalty": None,
        "eos_token_id": 2,
    }    
}

In [16]:
topic_recommender_prompt = """
당신은 기술 전문가로 {topic}에 대한 블로그 게시물을 작성하려 합니다. 당신이 작성할 {topic} 블로그 게시물 주제를 {number}개 작성해 주세요. 중복된 주제는 제외해야 합니다.

예시: 
- 대규모 모델을 Amazon SageMaker Studio 노트북의 대화형 환경에서 파인튜닝하기
- torchserve로 딥러닝 모델을 쉽고 빠르게 배포하기

주제: 
"""

recommend_topic_prompt = PromptTemplate(
    input_variables=['topic', 'number'],
    template=topic_recommender_prompt,
    model_kwargs=PARAMS[MODEL_NAME]
)

final_prompt = recommend_topic_prompt.format(topic="머신러닝", number=5)
print(colored(final_prompt, 'green'))
print(llm(final_prompt))


당신은 기술 전문가로 머신러닝에 대한 블로그 게시물을 작성하려 합니다. 당신이 작성할 머신러닝 블로그 게시물 주제를 5개 작성해 주세요. 중복된 주제는 제외해야 합니다.

예시: 
- 대규모 모델을 Amazon SageMaker Studio 노트북의 대화형 환경에서 파인튜닝하기
- torchserve로 딥러닝 모델을 쉽고 빠르게 배포하기

주제: 

1) 머신 러닝 프로젝트 관리 도구(프로젝트 관리자, 코드 빌더 등)와의 통합	2) 머신 러닝 모델 학습 및 훈련을 위한 최고의 도구들과의 통합 3) 머신 러닝 모델 구축 및 테스트를 위한 최고의 프레임워크 4) 머신 러닝 모델 구축 및 테스트를 위한 최고의 라이브러리 5) 머신 러닝 모델 구축 및 테스트를위한 최고의 툴킷 제공합니다.
